In [3]:
pip install bs4

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/143.0 kB ? eta -:--:--
     --------                                30.7/143.0 kB 1.4 MB/s eta 0:00:01
     --------                                30.7/143.0 kB 1.4 MB/s eta 0:00:01
     ----------------------------         112.6/143.0 kB 939.4 kB/s eta 0:00:01
     ----------------------------         112.6/143.0 kB 939.4 kB/s eta 0:00:01
     ----------------------------         112.6/143.0 kB 939.4 kB/s eta 0:00:01
     ------------------------------------ 143.0/143.0 kB 567.3 kB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1266 sha256=947fe9dfdd20c383c5bf4223e9724cad16f3957

In [17]:
import pandas as pd
import time
import json
import urllib
import json
import selenium

from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service


In [30]:
service = Service(executable_path='C:/ProgramData/Microsoft/Windows/Start Menu/Programs/chromedriver.exe')
driver = webdriver.Chrome(service=service)

def openbrowser(locid, key):
    driver.wait = WebDriverWait(driver, 5)
    driver.maximize_window()
    words = key.split()
    txt =''    
    for w in words:
        txt +=(w+'+')
    #print (txt)
    driver.get("https://www.glassdoor.co.in/Job/jobs.htm?suggestCount=0&suggestChosen=true&clickSource=searchBtn&typedKeyword={}\
               &sc.keyword={}&locT=C&locId={}&jobType=fulltime&fromAge=1&radius=6&cityId=-1&minRating=0.0&industryId=-1\
               &sgocId=-1&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0".format(txt[:-1],txt[:-1], locid))
    return driver

def geturl(driver):
    url = set()
    while True:
        print(len(url))
        if len(url)>=20:
            break
        soup1 = BeautifulSoup(driver.page_source, "html.parser")
        
        main = soup1.find_all("li",{"class":"jl"})
        
        for m in main:
            url.add('https://www.glassdoor.co.in{}'.format(m.find('a')['href']))       
        try:
            next_element = soup1.find("li", {"class": "next"})
            try:
                next_exist = next_element.find('a')
            except AttributeError:
                driver.quit()
                break
            except NoSuchElementException:
                driver.quit()
                break
            if next_exist:
    
                driver.find_element_by_class_name("next").click()
                time.sleep(2)
            else:
                driver.quit()
                break
        except ElementClickInterceptedException:
            pass
        
    return list(url)

x =openbrowser(locid =4477468, key='"software testing"')
with open('url_software_testing_loc_bangalore.json','w') as f:
    json.dump(geturl(driver),f, indent = 4)
    print("file created")

0
file created


In [1]:
with open('url_software_testing_loc_bangalore.json','r') as f:
    url = json.load(f)
data ={}    
i = 1
jd_df = pd.DataFrame()

service = Service(executable_path='C:/ProgramData/Microsoft/Windows/Start Menu/Programs/chromedriver.exe')
driver = webdriver.Chrome(service=service)

for u in tqdm(url):
    driver.wait = WebDriverWait(driver, 2)
    driver.maximize_window()
    driver.get(u)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
       
        header = soup.find("div",{"class":"header cell info"})
        position = driver.find_element_by_tag_name('h2').text
        company = driver.find_element_by_xpath("//span[@class='strong ib']").text
        location = driver.find_element_by_xpath("//span[@class='subtle ib']").text
        jd_temp = driver.find_element_by_id("JobDescriptionContainer")
        jd = jd_temp.text
        info = soup.find_all("infoEntity")
    except IndexError:
        print('IndexError: list index out of range')
    except NoSuchElementException:
        pass
    data[i] = {
        'url' :u,
        'Position':position,
        'Company': company,
        'Location' :location,
        'Job_Description' :jd
    }
    i+=1     
driver.quit()
jd_df = pd.DataFrame(data)
jd = jd_df.transpose()

jd = jd[['url','Position','Company','Location','Job_Description']]
jd.to_csv('jd_software_testing_banglore.csv')
print('file created')

FileNotFoundError: [Errno 2] No such file or directory: 'url_software_testing_loc_bangalore.json'